In [ ]:
import random
from typing import List, Dict, Any
from dataclasses import dataclass

# Corrected: Import HFModel from its specific submodule
from dspy.models.hf import HFModel

# Import other necessary DSPy components
from dspy import (
    ChainOfThought,
    Signature,
    InputField,
    OutputField,
    Module,
    Prediction,
    configure,
    Example
)

# --- LLM Configuration ---
# 1. Configure the Language Model (LLM)
# This will now use the locally cached 'google/flan-t5-small' model.
# No download will occur here if you've run the download_model.py script.
llm = HFModel(model='google/flan-t5-small')

# 2. Configure DSPy to use this LLM
configure(lm=llm)

# --- Data and Corpus ---
MY_CORPUS_PASSAGES = [
    "1) Maximize the Beauty - fully channel the beauty with in. Maybe ask what makes this moment beautiful? See if beauty can be increased in every situation. MtB also could be taken as a use of reason and also a disciplining of the senses to focus on beauty (i.e. all the pretty flowers, all the pretty birds). (Update 3-16-21 -This is one of the first 4 points created because they were on the top of my mind when I first started this project. Maximize the Beauty and Full Expression still are some of the concepts I am most influenced by, create an ethos around, consider to be primary to my concept of virtue, and aim to implement as much as possible.) ",
    "2) Full Expression - it takes a lot of effort for one to understand who they are when they are comfortable and how to channel the most real expressions of themselves what holds people back? Shyness, distraction (inability to focus on that which they want express)",
    "3) Expect Rising - this means our expectations are constantly rising. Kind of in line with \"give em an inch they'll take a mile\" also related to law of diminishing returns -\"The law of diminishing returns states that in all productive processes, adding more of one factor of production, while holding all others constant (\"ceteris paribus\"), will at some point yield lower incremental per-unit returns.\" Tooo much ice cream too much cash",
    "4) The Power of Pettiness - is the idea that pettiness is the destroyer of all people. That even the best of us can't truly be unaffected by the petty bullshit around us. They can however minimize its effects. In my own life I am obliterated by friends and coworker's snide remarks and judgments. How do I minimize its effects? By talking myself down, deep breaths, weed, alcohol, revenge...(this is the end of the first four points that were the strongest when I first started making these ideas on a small scrap of paper from the sides on union film sets.)",
    "5) Various meditation - I am a big fan of meditation techniques that help train the mind from moment to moment. There are various techniques that help provide concentration and awareness in the passing moments within shorter intervals than traditional formal meditation. While using a timer to make sure you meditate for at least 10 minutes is helpful. A stopwatch to monitor your awareness of time and to also use time to monitor your concentration and awareness. A and C are essential to have Full Expression. Update 2-26-19 - Passive observation ala Krishnamurti. Just chill. Dont do anything. Just observe whatever is present. Don't react to thoughts just observe them. Don't interact with them. If they are mind blowing write them down. Then return to just chilling and observing."
]

# --- Signatures for DSPy Modules ---

class GenerateAnswer(Signature):
    """Answer questions based on the provided context of personal philosophical ideas."""
    context = InputField(desc="Relevant personal philosophical ideas.")
    question = InputField(desc="The user's question.")
    answer = OutputField(desc="A thoughtful answer that aligns with the provided context.")

class IdeaConsistencyJudge(Signature):
    """Judge if the answer is consistent with the personal philosophical ideas and concepts in the context."""
    context = InputField(desc="Philosophical context from personal ideas corpus.")
    question = InputField(desc="Question about life philosophy or personal growth.")
    answer = InputField(desc="Answer to evaluate for consistency.")
    consistent_with_ideas: bool = OutputField(desc="Is the answer consistent with the philosophical ideas in the context?")
    explanation = OutputField(desc="Detailed reasoning for why the answer is or isn't consistent with the ideas.")

# --- DSPy Modules ---

# Create the judge with Chain of Thought reasoning
judge = ChainOfThought(IdeaConsistencyJudge)

class MyIdeasRAGProgram(Module):
    """RAG program that retrieves from personal ideas corpus and includes context in predictions."""
    def __init__(self, corpus_passages: List[str]):
        super().__init__()
        self.corpus = corpus_passages
        self.generator = ChainOfThought(GenerateAnswer)

    def retrieve_relevant_ideas(self, question: str, top_k: int = 2) -> List[str]:
        """Simple keyword-based retrieval."""
        keywords = question.lower().split()
        scored_passages = []
        for passage in self.corpus:
            score = sum(1 for keyword in keywords if keyword in passage.lower())
            scored_passages.append((score, passage))
        
        scored_passages.sort(key=lambda x: x[0], reverse=True)
        return [passage for _, passage in scored_passages[:top_k]]

    def forward(self, question: str):
        relevant_ideas = self.retrieve_relevant_ideas(question)
        context = "\n\n".join(relevant_ideas)
        prediction = self.generator(context=context, question=question)
        
        return Prediction(
            answer=prediction.answer,
            context=context,
            retrieved_ideas=relevant_ideas
        )

# --- Evaluation Logic ---

def idea_consistency_metric(example: Example, pred: Prediction) -> dict:
    """Evaluate how well an answer aligns with the personal philosophy corpus."""
    judgment = judge(
        context=example.context,
        question=example.question,
        answer=pred.answer
    )
    score = 1.0 if judgment.consistent_with_ideas else 0.0
    return {
        'consistent': judgment.consistent_with_ideas,
        'explanation': judgment.explanation,
        'score': score
    }

# --- Main Application Pipeline ---

class IdeasConsistencyPipeline:
    """A pipeline for evaluating answers against a personal philosophy."""
    def __init__(self, corpus_passages: List[str]):
        self.rag_program = MyIdeasRAGProgram(corpus_passages)

    def evaluate_answer(self, question: str, answer: str, context: str):
        """Evaluate a single answer against the ideas corpus."""
        eval_example = Example(question=question, context=context)
        prediction = Prediction(answer=answer)
        return idea_consistency_metric(eval_example, prediction)

    def interactive_mode(self):
        """Interactive mode for testing custom questions."""
        print("\n\n🎮 Interactive Ideas Consistency Checker")
        print("Ask questions about life philosophy and get evaluated answers!")
        print("Model: google/flan-t5-small")
        print("Type 'quit' to exit\n")

        while True:
            question = input("💭 Your question: ").strip()
            if question.lower() == 'quit':
                print("Exiting interactive mode. Goodbye! 👋")
                break
            if not question:
                continue

            pred = self.rag_program(question)
            print(f"\n🤖 Answer: {pred.answer}")

            result = self.evaluate_answer(question, pred.answer, pred.context)

            print(f"\n✅ Consistent with your ideas: {result['consistent']}")
            print(f"📊 Consistency score: {result['score']}")
            print(f"💭 Judge's reasoning: {result['explanation']}")
            print(f"📚 Context used: {pred.context[:150]}...")
            print("-" * 60)

# --- Execution ---

if __name__ == "__main__":
    pipeline = IdeasConsistencyPipeline(MY_corpus_passages)
    pipeline.interactive_mode()

ImportError: cannot import name 'HFModel' from 'dspy' (unknown location)

In [ ]:
from huggingface_hub import snapshot_download

# The Hugging Face model we want to use
MODEL_ID = "google/flan-t5-small"

def download_hf_model():
    """
    Downloads and caches the model and tokenizer files from Hugging Face.
    """
    print(f"📥 Starting download for model: {MODEL_ID}")
    
    # This function downloads the entire model repository to the local cache.
    # The cache is typically located at ~/.cache/huggingface/hub/
    snapshot_download(repo_id=MODEL_ID)
    
    print(f"✅ Download complete. Model '{MODEL_ID}' is cached locally.")

if __name__ == "__main__":
    # To run this script and download the model, open your terminal and type:
    # pip install huggingface_hub
    # python download_model.py
    download_hf_model()

📥 Starting download for model: google/flan-t5-small


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


README.md: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

(…)66c868570469993386a093bca18aff3c1e13b065:   0%|          | 0.00/308M [00:00<?, ?B/s]

(…)4d0ad38fca5173b09393b22b864ab8a55ba03d7c:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]